# Machine Learning Engineer Nanodegree
## Capstone Project
## Mushroom Classification

There exist many edible, high quality mushrooms which are rich with vitamins and minerals and have great value at market. However, some of the mushrooms are toxic, which can cause different type of health problems if consumed, and a small number of them is even deadly. Throughout this project, we want to classify the mushrooms and find out which ones are edible, and which ones are toxic.

We will analyse a dataset containing mushroom information, and train a few different supervised learning algorithms in order to classify the new inputs as poisonous or edible. We will run unsupervised learning techniques in order to see what kind of trait correlation exists between the mushroom, for the sake of improving our knowledge for feature selection and transformation. In the end, we will test and tune our supervised learning algorithms, in order to see which one is giving the best performance.

## Exploring the Data
Let's run the code cell below to load necessary Python libraries and load the mushroom data.

In [195]:
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import accuracy_score

mushroom_data = pd.read_csv("mushroom_dataset.csv")
print "Mushroom data read successfully!"

Mushroom data read successfully!


### Data Exploration
Let's begin by investigating our dataset. How many instances there are in the dataset, how many of them are poisonous, and how many of them are edible.

In [196]:
number_of_mushrooms = len(mushroom_data.count(axis=1))

number_of_features = len(mushroom_data.count(axis=0))-1

number_of_edible = len(mushroom_data[mushroom_data.type == "e"])

number_of_poisonous = len(mushroom_data[mushroom_data.type == "p"])

edible_percentile = (number_of_edible/float(number_of_mushrooms))*100

poisonous_percentile = 100-edible_percentile

print "Total number of mushrooms: {}".format(number_of_mushrooms)
print "Number of features: {}".format(number_of_features)
print "Number of edible mushrooms: {}".format(number_of_edible)
print "Number of poisonous mushrooms: {}".format(number_of_poisonous)
print "{:.2f}% of mushrooms are edible".format(edible_percentile)
print "{:.2f}% of mushrooms are poisonous".format(poisonous_percentile)

Total number of mushrooms: 8124
Number of features: 22
Number of edible mushrooms: 4208
Number of poisonous mushrooms: 3916
51.80% of mushrooms are edible
48.20% of mushrooms are poisonous


In [197]:
# Additional misc statistics

def count_habitat(t, h):
    return len(mushroom_data[np.all([mushroom_data.type==t, mushroom_data.habitat==h], axis=0)])

# How many different mushrooms live in different habitats
for t in ["e", "p"]:
    for h in ["g", "l", "m", "p", "u", "w", "d"]:
        print "There are {} {} mushrooms living in {}".format(count_habitat(t,h), t, h)

There are 1408 e mushrooms living in g
There are 240 e mushrooms living in l
There are 256 e mushrooms living in m
There are 136 e mushrooms living in p
There are 96 e mushrooms living in u
There are 192 e mushrooms living in w
There are 1880 e mushrooms living in d
There are 740 p mushrooms living in g
There are 592 p mushrooms living in l
There are 36 p mushrooms living in m
There are 1008 p mushrooms living in p
There are 272 p mushrooms living in u
There are 0 p mushrooms living in w
There are 1268 p mushrooms living in d


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Missing values
We know that missing values are noted as '?' in the dataset. Let's examine how many missing values we have and for which features is that the case

In [198]:
def detect_missing_values(data):
    from sets import Set
    
    df = pd.DataFrame(index = data.index)
    features_with_missing_values = Set()
    number_of_missing_values = 0
    
    for col, col_data in data.iteritems():
        
        missing_values_in_column = len(data[col][col_data=='?'])
        
        if missing_values_in_column > 0:
            features_with_missing_values.add(col)
            number_of_missing_values+=missing_values_in_column
        
        
    print "There are {} missing values from the following features: {}".format(number_of_missing_values, features_with_missing_values)
        
    
    
detect_missing_values(mushroom_data)

There are 2480 missing values from the following features: Set(['stalk-root'])


We can see that our missing values are contained only in stalk-root feature. We will see that in 'Feature Selection' chapter that 'stalk-root' feature can be predicted based on the other features, thus being unnecessary. We will remove it along with other unnecessary features in 'Feature Selection' part.

### Label encoding
In order to perform predictions, we will need to turn our data into numeric, and for that we will be using label encoder which is a part of sklearn

In [199]:
from sklearn import preprocessing

mushroom_data = mushroom_data.apply(preprocessing.LabelEncoder().fit_transform)

### Identify feature and target columns
Now, let's split our target column from feature columns.

In [200]:
target_column = mushroom_data.columns[0]
feature_columns = mushroom_data.columns[1:]

print "Feature columns:\n{}".format(feature_columns)
print "\nTarget column: {}".format(target_column)

X_all = mushroom_data[feature_columns]
y_all = mushroom_data[target_column]

print "\nFeature values:"
print X_all.head()

Feature columns:
Index([u'cap-shape', u'cap-surface', u'cap-color', u'bruises', u'odor',
       u'gill-attachment', u'gill-spacing', u'gill-size', u'gill-color',
       u'stalk-shape', u'stalk-root', u'stalk-surface-above-ring',
       u'stalk-surface-below-ring', u'stalk-color-above-ring',
       u'stalk-color-below-ring', u'veil-type', u'veil-color', u'ring-number',
       u'ring-type', u'spore-print-color', u'population', u'habitat'],
      dtype='object')

Target column: type

Feature values:
   cap-shape  cap-surface  cap-color  bruises  odor  gill-attachment  \
0          5            2          4        1     6                1   
1          5            2          9        1     0                1   
2          0            2          8        1     3                1   
3          5            3          8        1     6                1   
4          5            2          3        0     5                1   

   gill-spacing  gill-size  gill-color  stalk-shape   ...     \
0

### Feature selection
Since we have 22 features for every record in the dataset, let's analyse which features are irrelevant and can be removed from it. We will run a simple decision tree classifier on every feature and report how good the prediction is. Those features that can already be predicted, are not necessary for our target prediction.

In [201]:
def test_feature_accuracy(X):
    
    for col, col_data in X.iteritems():
        target_feature = X[col]
        new_x = X.drop(col, axis=1)
    
        from sklearn.cross_validation import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(new_x, target_feature, test_size=0.25, random_state=42)

        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(random_state=42)
        classifier.fit(X_train, y_train)

        score = classifier.score(X_test, y_test)
        
        if score>0.9:
            X = X.drop(col, axis=1)

        print "{} : {}".format(col, score)
        
    return  X
    
    
X_all = test_feature_accuracy(X_all)

cap-shape : 0.101920236337
cap-surface : 0.232890201871
cap-color : 0.110782865583
bruises : 1.0
odor : 0.70113244707
gill-attachment : 0.997045790251
gill-spacing : 0.981290004924
gill-size : 1.0
gill-color : 0.238306253077
stalk-shape : 1.0
stalk-root : 1.0
stalk-surface-above-ring : 0.663220088626
stalk-surface-below-ring : 0.65780403742
stalk-color-above-ring : 0.439684884293
stalk-color-below-ring : 0.439684884293
veil-type : 1.0
veil-color : 0.977843426883
ring-number : 1.0
ring-type : 1.0
spore-print-color : 0.563269325455
population : 0.374692269818
habitat : 0.472673559823


After removing those cells that can be predicted with score greater than 0.9, we now have 12 features and the number of features is significally reduced.

### Testing PCA for additional dimensionality reduction
Now, we can use PCA for additional dimensionality reduction. Let's see how many components will give us decent explained variance ratio.

In [202]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(X_all)

print np.sum(pca.explained_variance_ratio_)

0.926939376005


After a bit testing, it is possible to see that we can reduce the number of features to 8 principal components which will result in 0.9269 explained variance ratio. We can use this to turn this problem into binary clustering, and then try to minimise the error. Since we will turn our feature values into continuous values, it will be hard to achieve 100% with this type of problem we are tackling. We will first try to perform traditional supervised learning, and then if there is need to speed up the training/testing time as well as reduce variance and bias, we can come back to PCA dimensionality reduction and reduce the number of features for that sake.

## Training and Evaluating Models
Now, our data is prepared and we can begin modeling, training and evaluating our models. We will choose 3 supervised learning models, and try fitting our data to them. Then, we will see how do they perform and tune them accordingly. For those classifiers that have class_weight and sample_weight parameters, if there is need, we will tune them in order to implement cost sensitive learning (for example, it is better to predict edible mushroom as poisonous than poisonous mushroom as edible). We will evaluate our model with accuracy score and produce the tables which show the training set size, time, prediction time, accuracy score on training and accuracy score on testing set. For this problem, we will choose the following three algorithms:

- SVM
- Stochastic Gradient Descent
- Random Forest

- Support Vector Machines

The reason I choose SVMs is that the real world application of them is binary classification. Since we are dealing with binary classification in this problem, we will expect this model to perform well. The idea behind SVM is to find the greatest margin between two different sets. It is computing it mathematically to find the hyperplane which will separate the data with the greatest margin. If the data is not linearly separable in 2D, it will hit up higher dimension and then try to separate it. SVM performs poorly with large datasets and with datasets with lots of noise. This is not the case with our problem, and thus we can expect good results.

- Stochastic Gradient Descent

The Stochastic Gradient Descent is based on neural networks and the best way to use it is in handwriting recognition and such problems. This model should not be a good fit for our type of problem, but we will test it to compare its results with the other two. The algorithm is based on estimating the gradient by computing the part of the gradient for a small sample of randomly chosen training inputs.By averaging over this small sample it turns out that we can quickly get a good estimate of the true gradient, and this helps speed up gradient descent, and thus learning. Pros are that it is a fairly well studied algorithm, so for the most part the problems with GD have solutions. However, sometimes calculating the gradient can be very expensive or intractable if the size of our data is large.

- Random Forest

While reading papers regarding this problem we are tackling, we have seen that the decision trees can give very high accuracy scores while predicting the mushrooms as edible or poisonous. Random forest is an ensemble of decision trees. Each decision tree is constructed by using a random subset of the training data. After we have trained our forest, we can pass each test row through it, in order to output a prediction. This model should perform the best because it is based on decision trees, and decision trees is very good fit for this kind of problem

### Setup

Let's create three helper functions which we will be using for training and testing our models. 
Create confusion matrix function is pretty straightforward. It creates the confusion matrix based on true and predicted values. 
Train classifier function will take a classifier as a param and train it with the provided data. 
Predict labels will take as input fit classifier, features and a target labeling. Then it will make predictions using the accuracy score. 
Train predict will take as input a classifier, training and testing data and it will perform train classifier and predict labels.

In [204]:
def create_confusion_matrix(true, pred):
    from sklearn.metrics import confusion_matrix
    print "----------------"
    print "Confusion matrix\n"
    print confusion_matrix(true, pred)
    print "\n----------------\n"

def train_classifier(clf, X_train, y_train):

    start = time()
    clf.fit(X_train, y_train)
    end = time()
    print "Trained model in {:.4f} seconds".format(end - start)
    
def predict_labels(clf, features, target, confusion_matrix=False):

    start = time()
    y_pred = clf.predict(features)
    end = time()

    print "Made predictions in {:.4f} seconds.".format(end - start)
    
    if confusion_matrix:
        create_confusion_matrix(target.values, y_pred)
    
    return accuracy_score(target.values, y_pred)


def train_predict(clf, X_train, y_train, X_test, y_test, confusion_matrix=False):

    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    train_classifier(clf, X_train, y_train)
    
    print "Accuracy score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train, confusion_matrix))
    print "Accuracy score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test, confusion_matrix))

### Implementation: Model Performance Metrics
Now we will import our supervised learning models and run the train_predict function for each one. We will use different training set sizes (100, 200, 300). Details such as prediction time, accuracy scores, training time etc. will be contained in the tabular results below.

In [208]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

clf_A = SVC(random_state=42)
clf_B = SGDClassifier(random_state=42)
clf_C = RandomForestClassifier()

for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0026 seconds
Made predictions in 0.0005 seconds.
Accuracy score for training set: 1.0000.
Made predictions in 0.0079 seconds.
Accuracy score for test set: 0.8922.
Training a SVC using a training set size of 200. . .
Trained model in 0.0048 seconds
Made predictions in 0.0013 seconds.
Accuracy score for training set: 0.9900.
Made predictions in 0.0098 seconds.
Accuracy score for test set: 0.9222.
Training a SVC using a training set size of 300. . .
Trained model in 0.0055 seconds
Made predictions in 0.0027 seconds.
Accuracy score for training set: 0.9933.
Made predictions in 0.0157 seconds.
Accuracy score for test set: 0.9473.

SGDClassifier: 

Training a SGDClassifier using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0002 seconds.
Accuracy score for training set: 0.5900.
Made predictions in 0.0003 seconds.
Accuracy score for test set: 0.5574.
Training a SGDClassifier

### Tabular Results

** Classifer 1 - SVC **  

| Training Set Size | Training Time | Prediction Time (test) | Accuracy Score (train) | Accuracy Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0012                  | 0.0005                 |   1.0000         | 0.8922          |
| 200               | 0.0032                  | 0.0011                 |   0.9900         | 0.9222          |
| 300               | 0.0044                  | 0.0026                 |   0.9933         | 0.9473          |

** Classifer 2 - Stochastic Gradient Descent Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | Accuracy Score (train) | Accuracy Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0006                  | 0.0001                 | 0.5900           | 0.5574          |
| 200               | 0.0006                  | 0.0001                 | 0.6650           | 0.6539          |
| 300               | 0.0006                  | 0.0002                 | 0.8500           | 0.8518          |

** Classifer 3 - Random Forests Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | Accuracy Score (train) | Accuracy Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0237                  | 0.0009                 | 1.0000           | 0.9581          |
| 200               | 0.0228                  | 0.0009                 | 1.0000           | 0.9695          |
| 300               | 0.0227                  | 0.0010                 | 1.0000           | 0.9665          |

## Choosing the Best Model
Based on the experiments performed earlier, we have confirmed our assumptions about each model. Random Forests performs the best, as it is ensemble of decision trees. SVMs are performing well as we are dealing with the binary classification here, and Stochastic Gradient Descent is the fastest, but the accuracy score is very low compared to the other two models.

### Model Tuning
Now, it is time to apply first iteration of tuning our Random Forests model. We will be using grid search (`GridSearchCV`).

In [139]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {'max_depth': [2,4,6,10,15], 'min_samples_leaf': [2,5,10], 'max_features':[1,2,10]}
 
clf = RandomForestClassifier(random_state=42)

accuracy_scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(clf, parameters, scoring=accuracy_scorer)

grid_obj = grid_obj.fit(X_train, y_train)

clf = grid_obj.best_estimator_

print "Tuned model has a training accuracy score of {:.4f}.".format(predict_labels(clf, X_train, y_train, True))
print "Tuned model has a testing accuracy score of {:.4f}.".format(predict_labels(clf, X_test, y_test, True))

Made predictions in 0.0044 seconds.
----------------
Confusion matrix

[[3168    0]
 [   0 2925]]

----------------

Tuned model has a training accuracy score of 1.0000.
Made predictions in 0.0020 seconds.
----------------
Confusion matrix

[[1040    0]
 [   0  991]]

----------------

Tuned model has a testing accuracy score of 1.0000.


### Confusion Matrices

- Confusion Matrix for training

| True Positive | True Negative | False Positive | False Negative |
| :---------------:| :--------------------:| :---------------: | :---------------------: |
| 3168               | 2925                  | 0               | 0                  |


- Confusion Matrix for testing

| True Positive | True Negative | False Positive | False Negative |
| :---------------:| :--------------------:| :---------------: | :---------------------: |
| 1040               | 991                  | 0               | 0                  |


After tuning our Random Forest model, we have achieved exceptional results. We have successfully obtained 100% accuracy score on both training and testing sets with satisfiable prediction time of 0.0044 and 0.0020 seconds. Since we managed to to successfully classify 100% of mushrooms both in training and testing set, the additional dimensionality reduction with PCA is not necessary. Random Forests model is very quick and accurate.